# Learning with non-classical Logic

In [ ]:
using Pkg
Pkg.activate("..")
Pkg.instantiate()
Pkg.status()
Pkg.resolve()

In [ ]:
using Random
Random.seed!(1605)

## Learning with Modal Logic

In the cells below, we are going to...

TODO

In [ ]:
using ARFFFiles

using DataFrames
using MLJ
using Plots
using Random
using StatsBase
using SoleData
using SoleModels

In [ ]:
using SoleData.Artifacts: fillartifacts, load, NatopsLoader
fillartifacts()
# TODO load Natops from the dataset folder, using load_NATOPS
X, y = SoleData.Artifacts.load(NatopsLoader())

In [ ]:
X_ninstances, X_nattributes = size(X)
X_ndatapoints = length(X[1,1])

println("Number of instances: $(X_ninstances)")
println("Number of attributes: $(X_nattributes)")
println("Number of datapoints for each attribute: $(X_ndatapoints)")

In [ ]:
# for every combination of instance and attributes,
# we are still dealing with the same number of datapoints (51)
all(
    i -> length(X[i[1],i[2]]) == X_ndatapoints, 
    Iterators.product(1:X_ninstances, 1:X_nattributes)
)

In [ ]:
# try to change the target attribute
_attribute = 4
plot(X[1,_attribute], label = names(X)[_attribute])

In [ ]:
countmap(y)

In [ ]:
 # let us summarize one instance for each class
plot(map(i -> 
    plot(collect(X[i,:]), 
        labels=nothing,
        title=y[i]), 
        1:30:180
    )..., 
    layout = (2, 3), 
    size = (1500,400)
)

In [ ]:
# length of X[hand tip l] of the first instance 
length(X[1,1])

In [ ]:
# each instance can be shaped as a Kripke Frame, whose worlds encode all the intervals 
# in the range [1, 51] (including the degenerate, punctual cases such as [1, 1])
fr = SoleLogics.frame(X, 1)

In [ ]:
allworlds(fr) |> collect

In [ ]:
using SoleLogics: Interval

# enumerate the intervals that are "Later" than [1,10]
collect(accessibles(fr, Interval(1,10), IA_L))

In [ ]:
# we compute the value of a certain feature on each world where we can
feature = SoleData.VariableMax(4)

In [ ]:
plot(X[1, 4], labels="X Hand tip right")

In [ ]:
SoleData.featvalue(feature, X, 1, Interval(10, 30))

In [ ]:
# when we are interested in windowing the data, it is easy to transform a dataset into a 
# Kripke Model
Xk = scalarlogiset(X)

In [ ]:
# we can check custom conditions over the logiset we just created
p = Atom(ScalarCondition(feature, <, 1.0))
check(p, Xk, 1, Interval(10, 30))

In [ ]:
plot(collect(X[1, 4:6]), labels=["V4 (X right hand)" "V5 (Y right hand)" "V6 (Z right hand)"])

In [ ]:
p = Atom(ScalarCondition(VariableMin(4), >, 1.0))
q = Atom(ScalarCondition(VariableMax(5), <=, 3.0))
r = Atom(ScalarCondition(VariableMax(6), <=, 0.0))

phi = ¬p ∨ (q ∧ r)
println(syntaxstring(phi))

check(phi, SoleLogics.LogicalInstance(Xk, 1), Interval(10, 30))

Let us try to check some modal formulae.

In [ ]:
boxlater = box(SoleLogics.IA_A)

In [ ]:
later_always_phi = boxlater(phi)

In [ ]:
check(later_always_phi, SoleLogics.LogicalInstance(Xk, 1), Interval(10, 30))

In [ ]:
SoleLogics.getinstance(Xk, 1)

In [ ]:
# let us try with an even more complex scenario
check_mask = zeros(Int64, 51)
for i in 1:X_ndatapoints
    check_mask[i] = check(phi, SoleLogics.LogicalInstance(Xk, i), Interval(1,30))
end

println(check_mask)

### Modal Decision Trees

In [ ]:
using SoleBase
using ModalDecisionTrees

In [ ]:
# the experiment we are just going to execute could be too 
# heavy for standard commodity hardware;
# we can reduce data dimensionality via a moving window
X_small = broadcast(x -> movingwindow(mean, x; nwindows = 10, relative_overlap = 0.2), X)

X_small_ninstances, X_small_nattributes = size(X_small)
X_small_ndatapoints = length(X_small[1,1])

println("The number of datapoints changed from $(X_ndatapoints) to $(X_small_ndatapoints)")

In [ ]:
features = [maximum, minimum]
Xk_small = scalarlogiset(X_small, features)

In [ ]:
model = ModalDecisionTree(; relations = :IA, features = [minimum, maximum])

In [ ]:
(X_small_train, X_small_test), (y_small_train, y_small_test) = partition(
    (X_small, y), 0.7, rng=121, shuffle=true, multi=true);

In [ ]:
# bind the modal decision tree to the logiset;
# then train it and compute the accuracy

mach = machine(model, X_small_train, y_small_train)
@time fit!(mach);

y_small_predict_probabilities = MLJ.predict(mach, X_small_test)
y_small_predict = mode.(y_small_predict_probabilities)

MLJ.accuracy(y_small_predict, y_small_test)

In [ ]:
# show the restricted modal decision tree learned
printmodel(report(mach).rawmodel_full; hidemodality = true)

In [ ]:
# show its *pure* version
printmodel(report(mach).solemodel_full; show_metrics = true, hidemodality = true)

In [ ]:
simplified_restricted_tree = ModalDecisionTrees.prune(
    report(mach).rawmodel_full; simplify = true)

puretree = ModalDecisionTrees.translate(simplified_restricted_tree)
printmodel(
    puretree; 
    threshold_digits = 2, 
    use_feature_abbreviations = true, 
    parenthesize_atoms = false, 
    variable_names_map = [names(X)], 
    hidemodality = true
)

println("# Leaves: ", SoleModels.nsubmodels(puretree))
println("# Classes: ", length(unique(y)))

In [ ]:
# print the leaf rules and their training performances
ruleset = listrules(puretree)
printmodel.(
    ruleset; 
    show_metrics = true, 
    threshold_digits = 2, 
    use_feature_abbreviations = true, 
    parenthesize_atoms = false, 
    hidemodality = true
);

In [ ]:
println("IF\n\t", 
    SoleLogics.experimentals.formula2natlang(
        antecedent(ruleset[4]);
        threshold_digits = 2,
        variable_names_map = [names(X)]
    )
)

println("THEN\n\t", consequent(ruleset[4]))

### Modal Association Rules

Now, we follow an *unsupervised* approach, ignoring the class label.

The hypothesis here, is that a logical formula is *interesting*, if it happens 
to be frequently satisfied across all the instances of a dataset $\mathcal{I}$.

Given an alphabet $\mathcal{P}$ of propositional literals, the formula we are dealing with
are literal conjunctions called *itemsets*.

An itemset that is also frequent is called *frequent itemset*.

More formally, given a dataset $\mathcal{I}$, a propositional alphabet $\mathcal{P}$
and a minimum threshold $s$, a frequent pattern $\mathsf{P} \subseteq \mathcal{P}$ is such that

$$\frac{| \{I \in \mathcal{I} \mid I \models \mathsf{P} \} |}{|\mathcal{I}|} \geq s$$

The ratio above is called *support*.

In [ ]:
using ModalAssociationRules

In [ ]:
# these are just three toy atoms
p = Atom(ScalarCondition(VariableMax(4), >=, 2)) |> Item
q = Atom(ScalarCondition(VariableMin(5), <=, 1.5)) |> Item
r = Atom(ScalarCondition(VariableMax(6), >=, 0.0)) |> Item

In [ ]:
# one Itemset is just a wrapper around an optimized version of CONJUNCTION in Sole
pq = Itemset([p, q])
pr = Itemset([p, r])
qr = Itemset([q, r])
pqr = Itemset([p, q, r])

In [ ]:
supertype(LeftmostConjunctiveForm) |> supertype

In [ ]:
Atom |> supertype |> supertype |> supertype 

In [ ]:
formula(pq)

### Exercise:
Define your own `mysupport` function.

Its argument must be of type `SoleLogics.Formula`, `SoleData.AbstractLogiset` and `SoleLogics.AbstractWorld`.

We only want to consider the instances that were originally associated with the `I have command` class.

We want to treat the Kripke model as a degenerate propositional logiset.

Then compute the support of the following itemsets: `p`, `q`, `r`, `p ∧ q`, `p ∧ r`, `r ∧ q`, `p ∧ q ∧ r`. 

The support must be rounded to the second decimal digit.

Solution (Base64):
ZnVuY3Rpb24gbXlzdXBwb3J0KHBoaTo6RiwgWGs6OkwsIHdvcmxkOjpXKSB3aGVyZSB7CiAgICBGPDpTb2xlTG9naWNzLkZvcm11bGEsIAogICAgTDw6U29sZURhdGEuQWJzdHJhY3RMb2dpc2V0LCAKICAgIFc8OlNvbGVMb2dpY3MuQWJzdHJhY3RXb3JsZAp9CiAgICAKICAgIF9uaW5zdGFuY2VzID0gbmluc3RhbmNlcyhYaykKCiAgICBjaGVja19tYXNrID0gemVyb3MoSW50OCwgX25pbnN0YW5jZXMpCgogICAgQGluYm91bmRzIEBzaW1kIGZvciBpIGluIDE6X25pbnN0YW5jZXMgCiAgICAgICAgY2hlY2tfbWFza1tpXSA9IGNoZWNrKHBoaSwgWGssIGksIHdvcmxkKQogICAgZW5kCgogICAgcmV0dXJuIHJvdW5kKG1lYW4oY2hlY2tfbWFzayk7IGRpZ2l0cyA9IDIpCmVuZA==

In [ ]:
# Write your definition here

In [ ]:
for phi in [p, q, r, pq, pr, qr, pqr]
    println(
        mysupport(formula(phi), SoleData.slicedataset(Xk, 1:30), Interval(1, X_ndatapoints)))
end

In [ ]:
# each atom is called Item in the jargon
alphabet = Vector{Item}([p, q, r])

In [ ]:
# this is a list of metrics for establishing when a conjunction of literals
# or *itemset* (a set of items) is frequent
itemsetmeasures = [gsupport, 0.0, 0.2]

# these are our meaningfulness measures, for establishing whether an association rule 
# is interesting or not
arulemeasures = [
    (gconfidence, 0.0, 0.5),
    (glift, 0.5, 0.5)
]

After having found all the frequent patterns, we can partition each of them into two parts $\mathsf{P}, \mathsf{Q}$, such that $\mathsf{P} \cap \mathsf{Q} = \emptyset$.

The two subsets 

compute various statistical metrics between pairs of them, called *meaningfulness measures* in the jargon.

The if-then rule for which all the meaningfulness measures we consider are good, is called *association rule*.

## Learning with Many-Valued Logic

### Many-Expert Decision Trees

`ManyExpertDecisionTrees.jl` is still in development and has not been released
yet!

In [ ]:
using ManyExpertDecisionTrees
using ManyExpertDecisionTrees: build_tree, prune_tree, FL

In [ ]:
using SoleLogics.ManyValuedLogics

allexperts = (GodelLogic, LukasiewiczLogic, ProductLogic)

In [ ]:
using Combinatorics

# Compute all possible expert compbinations (with replacement)
expertcomb = begin
    c = Vector{Vector{FuzzyLogic}}()
    for i in 1:length(allexperts)
        append!(c, collect(Combinatorics.with_replacement_combinations(allexperts, i)))
    end
    c
end

In [ ]:
using RDatasets # used to load the iris dataset


data = RDatasets.dataset("datasets", "iris");

In [ ]:
# This is useful to read results later 
expertcombreadable = map(expertcomb) do experts
    result = ""
    for expert in experts
        if (expert === GodelLogic)
            result *= "G"
        end
        if (expert === LukasiewiczLogic)
            result *= "L"
        end
        if (expert === ProductLogic)
            result *= "P"
        end
    end

    return result
end;

In [ ]:
correct = [[0.0, 0.0] for _ in 1:length(expertcomb)];
wrong = [[0.0, 0.0] for _ in 1:length(expertcomb)];
vague = [[0.0, 0.0] for _ in 1:length(expertcomb)];

In [ ]:
X, y = begin
    X = data[:, 1:end-1]
    y = data[:, size(data, 2)]
    X, y
end

In [ ]:
n_runs = 10

for i in 1:n_runs
    # Partition set into training and validation
    X_train, y_train, X_test, y_test = begin
        train, test = partition(eachindex(y), 0.8, shuffle=true, rng=i)
        X_train, y_train = X[train, :], y[train]
        X_test, y_test = X[test, :], y[test]
        X_train, y_train, X_test, y_test
    end

    # Build a standard decision tree
    dt = build_tree(y_train, Matrix(X_train))
    dt = prune_tree(dt, 0.9)

    # For each expert combination, build a ManyExpertDecisionTree 
    Threads.@threads for k in eachindex(expertcomb)
        mf_experts = ntuple(_ -> FL.GaussianMF, length(expertcomb[k]))
        MXA = ManyExpertAlgebra(expertcomb[k]...)

        medt = manify(dt, X_train, mf_experts...)

        y_pred = map(eachrow(X_test)) do row
            result = ManyExpertDecisionTrees.apply(
                medt,
                MXA,
                Vector{Float64}(row)
            )
            return length(result) != 1 ? :vague : first(result)
        end

        # Extrapolating statistics
        n_total = length(y_test)

        n_vague = count(==(:vague), y_pred)
        pvague = (n_vague / n_total) * 100

        n_correct = count(i -> y_pred[i] == y_test[i], 1:n_total)
        pcorrect = (n_correct / n_total) * 100

        n_wrong = n_total - n_correct - n_vague
        pwrong = (n_wrong / n_total) * 100

        deltacorrect = (pcorrect - correct[k][1])
        correct[k][1] += deltacorrect / i
        correct[k][2] += deltacorrect * (pcorrect - correct[k][1])

        deltawrong = (pwrong - wrong[k][1])
        wrong[k][1] += deltawrong / i
        wrong[k][2] += deltawrong * (pwrong - wrong[k][1])

        deltavague = (pvague - vague[k][1])
        vague[k][1] += deltavague / i
        vague[k][2] += deltavague * (pvague - vague[k][1])

    end
end

In [ ]:
# Process results: extract means and compute standard deviations (sample std)
correct_mean = [x[1] for x in correct]
correct_std = [sqrt(x[2] / (n_runs - 1)) for x in correct]

wrong_mean = [x[1] for x in wrong]
wrong_std = [sqrt(x[2] / (n_runs - 1)) for x in wrong]

vague_mean = [x[1] for x in vague]
vague_std = [sqrt(x[2] / (n_runs - 1)) for x in vague]

df = DataFrame(
    experts=expertcombreadable,
    correct_mean=correct_mean,
    correct_std=correct_std,
    wrong_mean=wrong_mean,
    wrong_std=wrong_std,
    vague_mean=vague_mean,
    vague_std=vague_std
)